In [10]:
import pandas as pd
import networkx as nx
import plotly.express as px


In [11]:
df_nodes = pd.read_csv('nodes.csv')
df_edges = pd.read_csv('links.csv')

In [12]:

df_nodes.head()

,Unnamed: 0,type,country,ProductServices,PointOfContact,HeadOfOrg,founding_date,revenue,TradeDescription,_last_edited_by,_last_edited_date,_date_added,_raw_source,_algorithm,id,dob
0,0,Entity.Organization.Company,Uziland,Unknown,Rebecca Lewis,Émilie-Susan Benoit,1954-04-24,5994.73,Unknown,Pelagia Alethea Mordoch,2035-01-01,2035-01-01,Existing Corporate Structure Data,Automatic Import,"Abbott, Mcbride and Edwards",NaN
1,1,Entity.Organization.Company,Mawalara,Furniture and home accessories,Michael Lopez,Honoré Lemoine,2009-06-12,71766.67,Abbott-Gomez is a leading manufacturer and sup...,Pelagia Alethea Mordoch,2035-01-01,2035-01-01,Existing Corporate Structure Data,Automatic Import,Abbott-Gomez,NaN
2,2,Entity.Organization.Company,Uzifrica,Food products,Steven Robertson,Jules Labbé,2029-12-15,0.00,Abbott-Harrison is a leading manufacturer of h...,Pelagia Alethea Mordoch,2035-01-01,2035-01-01,Existing Corporate Structure Data,Automatic Import,Abbott-Harrison,NaN
3,3,Entity.Organization.Company,Islavaragon,Unknown,Anthony Wyatt,Dr. Víctor Hurtado,1972-02-16,0.00,Unknown,Pelagia Alethea Mordoch,2035-01-01,2035-01-01,Existing Corporate Structure Data,Automatic Import,Abbott-Ibarra,NaN
4,4,Entity.Organization.Company,Oceanus,Unknown,Tracy Schmidt,Jacques Le Roux de Leclercq,1954-04-06,4746.67,Unknown,Pelagia Alethea Mordoch,2035-01-01,2035-01-01,Existing Corporate Structure Data,Automatic Import,Abbott-Sullivan,NaN


In [13]:
df_edges.head()

,Unnamed: 0,start_date,type,_last_edited_by,_last_edited_date,_date_added,_raw_source,_algorithm,source,target,key,end_date
0,0,2016-10-29,Event.Owns.Shareholdership,Pelagia Alethea Mordoch,2035-01-01,2035-01-01,Existing Corporate Structure Data,Automatic Import,Avery Inc,"Allen, Nichols and Thompson",0,NaN
1,1,2035-06-03,Event.Owns.Shareholdership,Niklaus Oberon,2035-07-15,2035-07-15,Oceanus Corporations Monthly - Jun '35,Manual Entry,Berger-Hayes,"Jensen, Morris and Downs",0,NaN
2,2,2028-11-20,Event.Owns.Shareholdership,Pelagia Alethea Mordoch,2035-01-01,2035-01-01,Existing Corporate Structure Data,Automatic Import,Bowers Group,Barnett Inc,0,NaN
3,3,2024-09-04,Event.Owns.Shareholdership,Pelagia Alethea Mordoch,2035-01-01,2035-01-01,Existing Corporate Structure Data,Automatic Import,Bowman-Howe,Bennett Ltd,0,NaN
4,4,2034-11-12,Event.Owns.Shareholdership,Pelagia Alethea Mordoch,2035-01-01,2035-01-01,Existing Corporate Structure Data,Automatic Import,Boyd and Sons,"Armstrong, Bennett and Deleon",0,NaN


In [14]:
#converter para date founding_date, _last_edited_date e _date_added
df_nodes['founding_date'] = pd.to_datetime(df_nodes['founding_date'])
df_edges['start_date'] = pd.to_datetime(df_edges['start_date'])
df_edges['end_date'] = pd.to_datetime(df_edges['end_date'])

In [15]:
#max e min founding_date, start_date e end_date
print(df_nodes['founding_date'].max())
print(df_nodes['founding_date'].min())
print(df_edges['start_date'].max())
print(df_edges['start_date'].min())
print(df_edges['end_date'].max())
print(df_edges['end_date'].min())



2035-12-29 00:00:00
1945-01-01 00:00:00
2035-12-29 00:00:00
1952-05-31 00:00:00
2035-12-29 00:00:00
2035-01-01 00:00:00


In [16]:
def create_dynamic_graphs(df_nodes, df_edges, periods):
    graphs = {}
    for period in periods:
        G = nx.DiGraph()
        
        # Filtrar nós selecionados, se a lista selected_nodes estiver definida

        df_nodes_period = df_nodes[df_nodes['founding_date'] <= period]
        
        # Adicionar nós ao grafo G
        for _, row in df_nodes_period.iterrows():
            G.add_node(row['id'], **row.to_dict())
        
        # Filtrar e adicionar arestas com base em start_date e end_date
        df_edges_period = df_edges[(df_edges['start_date'] <= period) & 
                                   ((df_edges['end_date'].isna()) | (df_edges['end_date'] >= period))]
        
        for _, row in df_edges_period.iterrows():
            G.add_edge(row['source'], row['target'], **row.to_dict())
        
        graphs[period] = G
    
    return graphs

In [8]:
# Definindo os períodos de interesse
periods = pd.date_range(start='2032-01-01', end='2035-12-29', freq='Q').strftime('%Y-%m').tolist()

# Criando grafos para cada período
graphs = create_dynamic_graphs(df_nodes, df_edges, periods)

C:\Users\maico\AppData\Local\Temp\ipykernel_45848\3661856242.py:2: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  periods = pd.date_range(start='2032-01-01', end='2035-12-29', freq='Q').strftime('%Y-%m').tolist()


In [9]:
centrality_data = []
# Iterar sobre cada período e calcular as métricas de centralidade para o node específico
for period, G in graphs.items():
    # Calcular as métricas de centralidade
    eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    
    # Obter as métricas de centralidade específicas para o nodo 'SouthSeafood Express Corp'
    if 'SouthSeafood Express Corp' in G:
        node = 'SouthSeafood Express Corp'
        centrality_data.append({
            'period': period,
            'node': node,
            'eigenvector': eigenvector_centrality.get(node, 0),
            'degree': degree_centrality.get(node, 0),
            'betweenness': betweenness_centrality.get(node, 0)
        })

# Criar um DataFrame com os dados de centralidade específicos para 'SouthSeafood Express Corp'
centrality_df = pd.DataFrame(centrality_data)
print(centrality_df)


KeyboardInterrupt: 

In [ ]:
centrality_data = []

for period, G in graphs.items():
    eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    
    for node in G.nodes():
        centrality_data.append({
            'period': period,
            'node': node,
            'eigenvector': eigenvector_centrality.get(node, 0),
            'degree': degree_centrality.get(node, 0),
            'betweenness': betweenness_centrality.get(node, 0)
        })

centrality_df = pd.DataFrame(centrality_data)


In [17]:
#centrality_df.to_csv('centrality_new.csv', index=False)
centrality_df = pd.read_csv('centrality_new.csv')
centrality_df['period'] = pd.to_datetime(centrality_df['period'])

In [18]:
centrality_df

,period,node,eigenvector,degree,betweenness
0,1945-03-01,Coleman-Lindsey,3.333333e-01,0.000000,0.0
1,1945-03-01,Davis-Ramsey,3.333333e-01,0.000000,0.0
2,1945-03-01,Henderson LLC,3.333333e-01,0.000000,0.0
3,1945-03-01,Hernandez Group,3.333333e-01,0.000000,0.0
4,1945-03-01,Howell LLC,3.333333e-01,0.000000,0.0
...,...,...,...,...,...
2230079,2035-09-01,Braydon Mccarthy,4.355468e-08,0.000033,0.0
2230080,2035-09-01,Lauren Wagner,4.355468e-08,0.000017,0.0
2230081,2035-09-01,Emily Yates,4.355468e-08,0.000017,0.0
2230082,2035-09-01,Patrick Mueller,4.355468e-08,0.000017,0.0


In [19]:
# Filtrar para incluir apenas dados anuais (primeiro dia do ano)
#centrality_df = centrality_df[centrality_df['period'].dt.month == 1]

# Garantir que a coluna 'period' esteja no formato de string para plotly
#centrality_df['period'] = centrality_df['period'].dt.to_period('Y').astype(str)

In [20]:
centrality_df[centrality_df['node'] == 'SouthSeafood Express Corp']

,period,node,eigenvector,degree,betweenness
1891556,2033-03-01,SouthSeafood Express Corp,9.139300e-08,0.000000,0.0
1946127,2033-09-01,SouthSeafood Express Corp,9.186164e-08,0.000000,0.0
2002076,2034-03-01,SouthSeafood Express Corp,4.200923e-02,0.000017,0.0
2059437,2034-09-01,SouthSeafood Express Corp,4.196987e-02,0.000017,0.0
2118320,2035-03-01,SouthSeafood Express Corp,4.195030e-02,0.000017,0.0
2178541,2035-09-01,SouthSeafood Express Corp,9.301349e-01,0.000017,0.0


In [21]:
# Opcional: Filtrar para incluir apenas os nós mais importantes
centrality_df = centrality_df[centrality_df['period'] > '2004-01-01']
important_nodes = centrality_df.groupby('node')['eigenvector'].mean().sort_values(ascending=False).head(10).index
centrality_df_filtered = centrality_df[centrality_df['node'].isin(important_nodes)]


In [22]:
import plotly.express as px

# Visualização para Eigenvector Centrality
fig = px.line(centrality_df_filtered, x='period', y='eigenvector', color='node', title='Eigenvector Centrality Over Time')
fig.show()

# Visualização para Degree Centrality
fig = px.line(centrality_df_filtered, x='period', y='degree', color='node', title='Degree Centrality Over Time')
fig.show()

# Visualização para Betweenness Centrality
fig = px.line(centrality_df_filtered, x='period', y='betweenness', color='node', title='Betweenness Centrality Over Time')
fig.show()


In [38]:
df_centrality_old = pd.read_csv('centrality_after_before_change_ownership_seafood.csv')
df_centrality_old

,Unnamed: 0,Degree_Centrality_before,Eigenvector_Centrality_before,Betweenness Centrality_before,Degree_Centrality_after,Eigenvector_Centrality_after,Betweenness Centrality_after
0,"Abbott, Mcbride and Edwards",0.000132,2.463784e-18,8.430224e-05,0.0,1.973425e-16,0.0
1,Abbott-Gomez,0.000033,2.837294e-124,5.460767e-10,0.0,1.973425e-16,0.0
2,Abbott-Harrison,0.000099,7.742384e-07,1.347098e-02,0.0,1.973425e-16,0.0
3,Abbott-Ibarra,0.000083,6.161403e-104,3.385676e-08,0.0,1.973425e-16,0.0
4,Abbott-Sullivan,0.000116,2.363281e-21,2.527609e-04,0.0,1.973425e-16,0.0
...,...,...,...,...,...,...,...
60515,Braydon Mccarthy,0.000033,2.259908e-91,0.000000e+00,0.0,1.973425e-16,0.0
60516,Lauren Wagner,0.000017,4.611804e-92,0.000000e+00,0.0,1.973425e-16,0.0
60517,Emily Yates,0.000017,4.521315e-92,0.000000e+00,0.0,1.973425e-16,0.0
60518,Patrick Mueller,0.000017,6.555828e-92,0.000000e+00,0.0,1.973425e-16,0.0


In [40]:
#rename Unnamed: 0 to company
df_centrality_old = df_centrality_old.rename(columns={'Unnamed: 0':'Node'})
df_centrality_old = df_centrality_old.rename(columns={'Betweenness Centrality_before':'Betweenness_Centrality_before'})
df_centrality_old = df_centrality_old.rename(columns={'Betweenness Centrality_after':'Betweenness_Centrality_after'})


In [41]:
df_centrality_old

,Node,Degree_Centrality_before,Eigenvector_Centrality_before,Betweenness_Centrality_before,Degree_Centrality_after,Eigenvector_Centrality_after,Betweenness_Centrality_after
0,"Abbott, Mcbride and Edwards",0.000132,2.463784e-18,8.430224e-05,0.0,1.973425e-16,0.0
1,Abbott-Gomez,0.000033,2.837294e-124,5.460767e-10,0.0,1.973425e-16,0.0
2,Abbott-Harrison,0.000099,7.742384e-07,1.347098e-02,0.0,1.973425e-16,0.0
3,Abbott-Ibarra,0.000083,6.161403e-104,3.385676e-08,0.0,1.973425e-16,0.0
4,Abbott-Sullivan,0.000116,2.363281e-21,2.527609e-04,0.0,1.973425e-16,0.0
...,...,...,...,...,...,...,...
60515,Braydon Mccarthy,0.000033,2.259908e-91,0.000000e+00,0.0,1.973425e-16,0.0
60516,Lauren Wagner,0.000017,4.611804e-92,0.000000e+00,0.0,1.973425e-16,0.0
60517,Emily Yates,0.000017,4.521315e-92,0.000000e+00,0.0,1.973425e-16,0.0
60518,Patrick Mueller,0.000017,6.555828e-92,0.000000e+00,0.0,1.973425e-16,0.0


In [42]:
# Calcular a média das centralidades para cada nó
df_centrality_old['Degree_Centrality_mean'] = df_centrality_old[['Degree_Centrality_before', 'Degree_Centrality_after']].mean(axis=1)
df_centrality_old['Eigenvector_Centrality_mean'] = df_centrality_old[['Eigenvector_Centrality_before', 'Eigenvector_Centrality_after']].mean(axis=1)
df_centrality_old['Betweenness_Centrality_mean'] = df_centrality_old[['Betweenness_Centrality_before', 'Betweenness_Centrality_after']].mean(axis=1)

# Selecionar os top 10 nós baseados na média das centralidades
top_10_degree = df_centrality_old.sort_values(by='Degree_Centrality_mean', ascending=False).head(10)
top_10_eigenvector = df_centrality_old.sort_values(by='Eigenvector_Centrality_mean', ascending=False).head(10)
top_10_betweenness = df_centrality_old.sort_values(by='Betweenness_Centrality_mean', ascending=False).head(10)

# Combinar os top 10 nós em um DataFrame único, removendo duplicatas
top_nodes = pd.concat([top_10_degree, top_10_eigenvector, top_10_betweenness]).drop_duplicates().reset_index(drop=True)

# Exibir os top nós selecionados
print(top_nodes)


                           Node  Degree_Centrality_before  \
0                   Downs Group                  0.001983   
1                  Sandra Young                  0.001504   
2               Anne Villanueva                  0.001504   
3              Jennifer Gardner                  0.001504   
4                    Anna Davis                  0.001504   
5    Mills, Atkinson and Chavez                  0.001520   
6                 Melissa Davis                  0.001487   
7                 Joseph Gentry                  0.001487   
8                 Breanna Price                  0.001487   
9              Kenneth Gonzales                  0.001487   
10         Polák Procházka a.s.                  0.000000   
11             Catherine Macias                  0.001487   
12                   Erica Ford                  0.001487   
13                Kelsey Ortega                  0.001487   
14                  Sharon Moon                  0.000463   
15             Steven Ro

In [59]:
import plotly.graph_objects as go

# Função para criar gráficos comparativos de barras para os top nós
# Função para criar gráficos comparativos de barras para os nós principais
def plot_comparison(df, metric):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=df['Node'],
        y=df[f'{metric}_before'],
        name=f'{metric.replace("_", " ")} Antes'
    ))
    fig.add_trace(go.Bar(
        x=df['Node'],
        y=df[f'{metric}_after'],
        name=f'{metric.replace("_", " ")} Después'
    ))
    
    fig.update_layout(
        title=f'Comparación de {metric.replace("_", " ")} Antes y Después para los 10 Nodos Principales'
    )
    
    fig.show()

# Filtrar o DataFrame para apenas os top nós
def filter_top_nodes(df, top_nodes):
    return df[df['Node'].isin(top_nodes['Node'])]

# Filtrar os dados
top_degree_df = filter_top_nodes(df_centrality_old, top_10_degree)
top_eigenvector_df = filter_top_nodes(df_centrality_old, top_10_eigenvector)
top_betweenness_df = filter_top_nodes(df_centrality_old, top_10_betweenness)

# Plotar as comparações
plot_comparison(top_degree_df, 'Degree_Centrality')
plot_comparison(top_eigenvector_df, 'Eigenvector_Centrality')
plot_comparison(top_betweenness_df, 'Betweenness_Centrality')


In [44]:
# Calcular a média das centralidades para cada nó
df_centrality_old['Degree_Centrality_mean'] = df_centrality_old[['Degree_Centrality_before', 'Degree_Centrality_after']].mean(axis=1)
df_centrality_old['Eigenvector_Centrality_mean'] = df_centrality_old[['Eigenvector_Centrality_before', 'Eigenvector_Centrality_after']].mean(axis=1)
df_centrality_old['Betweenness_Centrality_mean'] = df_centrality_old[['Betweenness_Centrality_before', 'Betweenness_Centrality_after']].mean(axis=1)

# Selecionar os top 20 nós baseados na média das centralidades
top_20_degree = df_centrality_old.sort_values(by='Degree_Centrality_mean', ascending=False).head(20)
top_20_eigenvector = df_centrality_old.sort_values(by='Eigenvector_Centrality_mean', ascending=False).head(20)
top_20_betweenness = df_centrality_old.sort_values(by='Betweenness_Centrality_mean', ascending=False).head(20)

# Combinar os top 20 nós em um DataFrame único, removendo duplicatas
top_nodes = pd.concat([top_20_degree, top_20_eigenvector, top_20_betweenness]).drop_duplicates().reset_index(drop=True)

# Exibir os top nós selecionados
print(top_nodes)

                           Node  Degree_Centrality_before  \
0                   Downs Group                  0.001983   
1                  Sandra Young                  0.001504   
2               Anne Villanueva                  0.001504   
3              Jennifer Gardner                  0.001504   
4                    Anna Davis                  0.001504   
5    Mills, Atkinson and Chavez                  0.001520   
6                 Melissa Davis                  0.001487   
7                 Joseph Gentry                  0.001487   
8                 Breanna Price                  0.001487   
9              Kenneth Gonzales                  0.001487   
10                   Erica Ford                  0.001487   
11                Aaron Delgado                  0.001487   
12                  Joshua Kent                  0.001487   
13             Catherine Macias                  0.001487   
14                Kelsey Ortega                  0.001487   
15             Cynthia A

In [45]:
df_centrality_old[df_centrality_old['Node'] == 'Polák Procházka a.s.']

,Node,Degree_Centrality_before,Eigenvector_Centrality_before,Betweenness_Centrality_before,Degree_Centrality_after,Eigenvector_Centrality_after,Betweenness_Centrality_after,Degree_Centrality_mean,Eigenvector_Centrality_mean,Betweenness_Centrality_mean
60232,Polák Procházka a.s.,0.0,4.213984e-161,0.0,0.000562,0.705882,3.063491e-07,0.000281,0.352941,1.531745e-07


In [46]:
centrality_df[centrality_df['node'] == 'Polák Procházka a.s.']

,period,node,eigenvector,degree,betweenness
2178485,2035-09-01,Polák Procházka a.s.,0.000067,0.000563,0.0


In [47]:
centrality_df_top_degree = df_nodes[df_nodes['id'].isin(top_20_degree['Node'])]
centrality_df_top_eigenvector = df_nodes[df_nodes['id'].isin(top_20_eigenvector['Node'])]
centrality_df_top_betweenness = df_nodes[df_nodes['id'].isin(top_20_betweenness['Node'])]

In [48]:
top_nodes_cambiaron = pd.concat([
    centrality_df_top_degree['id'],
    centrality_df_top_eigenvector['id'],
    centrality_df_top_betweenness['id']
]).drop_duplicates().reset_index(drop=True).tolist()
#concatenar  os node dos dataframe em uma lista e eliminar os repetidos

In [49]:
top_nodes_cambiaron

['Downs Group',
 'Mills, Atkinson and Chavez',
 'Kelsey Ortega',
 'Joseph Gentry',
 'Cynthia Anderson',
 'Anna Davis',
 'Katie Evans',
 'Jennifer Gardner',
 'Wesley Phillips',
 'Melissa Davis',
 'Joshua Kent',
 'Aaron Delgado',
 'Breanna Price',
 'Sandra Young',
 'Alexandra Clay',
 'Kenneth Gonzales',
 'Erica Ford',
 'Catherine Macias',
 'Sydney Jones',
 'Anne Villanueva',
 'Alan Richardson',
 'Joseph Brown',
 'Jason Anderson',
 'Zachary Taylor',
 'Polák Procházka a.s.',
 'Abbott-Harrison',
 'Austin-Stephens',
 'Blankenship-Strickland',
 'Cervantes-Kramer',
 'Douglas, Hall and Pena',
 'Hart Ltd',
 'Holloway-Salas',
 'Johnson, Morales and Castro',
 'Lee-Fuentes',
 'Marsh Group',
 'Orr Inc',
 'Phelps, Montoya and Barnett',
 'Walker LLC',
 'Steven Robertson',
 'Sharon Moon',
 'Michael Howard DDS',
 'Annette Dennis',
 'Carlos Ferrell',
 'Janet Fox']

In [50]:
df_nodes_mas_cambio = df_nodes[df_nodes['id'].isin(top_nodes_cambiaron)]

In [51]:
df_nodes_mas_cambio['id'].unique()

array(['Abbott-Harrison', 'Austin-Stephens', 'Blankenship-Strickland',
       'Cervantes-Kramer', 'Douglas, Hall and Pena', 'Downs Group',
       'Hart Ltd', 'Holloway-Salas', 'Johnson, Morales and Castro',
       'Lee-Fuentes', 'Marsh Group', 'Mills, Atkinson and Chavez',
       'Orr Inc', 'Phelps, Montoya and Barnett', 'Walker LLC',
       'Steven Robertson', 'Kelsey Ortega', 'Joseph Gentry',
       'Cynthia Anderson', 'Anna Davis', 'Katie Evans',
       'Jennifer Gardner', 'Wesley Phillips', 'Alan Richardson',
       'Joseph Brown', 'Jason Anderson', 'Melissa Davis', 'Joshua Kent',
       'Aaron Delgado', 'Breanna Price', 'Sandra Young', 'Alexandra Clay',
       'Zachary Taylor', 'Kenneth Gonzales', 'Erica Ford',
       'Catherine Macias', 'Sydney Jones', 'Anne Villanueva',
       'Sharon Moon', 'Michael Howard DDS', 'Annette Dennis',
       'Carlos Ferrell', 'Janet Fox', 'Polák Procházka a.s.'],
      dtype=object)

In [52]:
periods = pd.date_range(start='2020-01-01', end='2023-12-31', freq='Q').strftime('%Y-%m').tolist()

# Criando grafos para cada período
graphs_mas_cambio = create_dynamic_graphs(df_nodes_mas_cambio, df_edges, periods)

C:\Users\maico\AppData\Local\Temp\ipykernel_45848\1264908247.py:1: FutureWarning:

'Q' is deprecated and will be removed in a future version, please use 'QE' instead.



KeyboardInterrupt: 

In [ ]:
centrality_mas_cambio = []

for period, G in graphs_mas_cambio.items():
    eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    
    for node in G.nodes():
        centrality_mas_cambio.append({
            'period': period,
            'node': node,
            'eigenvector': eigenvector_centrality.get(node, 0),
            'degree': degree_centrality.get(node, 0),
            'betweenness': betweenness_centrality.get(node, 0)
        })

centrality_mas_cambio_df = pd.DataFrame(centrality_mas_cambio)

#centrality_mas_cambio_df.to_csv('centrality_mas_cambio.csv', index=False)

NameError: name 'graphs_mas_cambio' is not defined

In [ ]:
import pandas as pd
import networkx as nx
import plotly.express as px


In [ ]:
#centrality_mas_cambio_df.to_csv('centrality_mas_cambio.csv', index=False)
centrality_mas_cambio_df = pd.read_csv('centrality_mas_cambio.csv')

In [ ]:
centrality_mas_cambio_df

In [ ]:
centrality_mas_cambio_df[centrality_mas_cambio_df['node'] == 'Polák Procházka a.s.']

In [ ]:
centrality_mas_cambio_df = centrality_mas_cambio_df[centrality_mas_cambio_df['node'].isin(top_nodes_cambiaron)]

In [ ]:
top_nodes_cambiaron

In [ ]:
# Visualização para Eigenvector Centrality
fig = px.line(centrality_mas_cambio_df, x='period', y='eigenvector', color='node', title='Eigenvector Centrality Over Time')
fig.show()


In [ ]:

# Visualização para Degree Centrality
fig = px.line(centrality_mas_cambio_df, x='period', y='degree', color='node', title='Degree Centrality Over Time')
fig.show()

In [ ]:
# Visualização para Betweenness Centrality
fig = px.line(centrality_mas_cambio_df, x='period', y='betweenness', color='node', title='Betweenness Centrality Over Time')
fig.show()